In [2]:
import models
import utils
import torch
import torch.nn.functional as f
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

torch.manual_seed(42)

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

Y = torch.tensor(np.load('../datasets/floored_exp_correl.npy'), dtype=dtype)
A_true = np.load('../datasets/students_correl.npy')
D_true = np.load('../datasets/questions_correl.npy')
# We assume we know the relevant concept of each question beforehand
concepts = np.nonzero(D_true)
num_students, num_concepts = A_true.shape
num_questions = D_true.shape[0]
guess_prob = 1/5

In [14]:
Y_rnn = Y.transpose(0,1)
Y_rnn.unsqueeze_(-1)
Y_rnn = Y_rnn.expand(num_questions,num_students,1)
Y_rnn.shape

torch.Size([100, 1000, 1])

## Training an RNN to predict students' performance

In [ ]:
hidden_size = 128
num_layers = 8
#extra params can be added: batch, bidirectional, bias...
init_params = (1, hidden_size, num_layers)
max_acc, best_params = utils.hyperparam_search(torch.nn.modules.rnn.LSTM, init_params,  {'lr':(-3,-1), 'reg':(-4,1)}, 10, 30, Y_rnn)
print('Best parameters: lr={}, reg={}, momentum={}'.format(*best_params))